In [3]:
import bilby
import tbilby
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from tbilby.core.prior.order_stats import TransdimensionalConditionalDescendingOrderStatPrior

In [4]:
x=np.linspace(0,150,1501)
mu=np.array([35, 74, 101])
# mock data
model=[]
mock_data=[]
sigma=[10,8,12]
A=[1.0,0.8,1.2]
for i in range(3):
    model.append(norm(mu[i],sigma[i]))
    mock_data.append(A[i]*model[i].pdf(x))
    
noise_model=norm(loc=0,scale=0.15)
np.random.seed(1234)
mock_noise=noise_model.rvs(1501)

data=mock_data[0]+mock_data[1]+mock_data[2]+mock_noise

In [5]:
n_peaks=6

In [6]:
def conversion_snr_to_amplitude(time, snr, sigma_g, sigma_noise):
    return snr*sigma_noise*np.sqrt(2 * np.sqrt(np.pi) * sigma_g * time[1] - time[0])
    

def gauss_pulse(time, SNR, mu, sigma_g, sigma):
    return  conversion_snr_to_amplitude(time, SNR, sigma_g, sigma) * norm(mu,sigma_g).pdf(time)

In [ ]:
component_functions_dict={}
component_functions_dict[gauss_pulse]=(n_peaks,'mu', 'SNR', 'sigma_g')

signal_model = tbilby.core.base.create_transdimensional_model('model',  component_functions_dict,\
                                                       returns_polarization=False,SaveTofile=False)

In [8]:
# Now lets instantiate a version of our GaussianLikelihood, giving it the time, data and signal model
time=np.linspace(0,150,1501)
likelihood = bilby.likelihood.GaussianLikelihood(time, data, signal_model)

In [9]:
# Here we define the priors
class TransdimensionalConditionalDescendingOrderStatPriorSNR(TransdimensionalConditionalDescendingOrderStatPrior):
   
    def transdimensional_condition_function(self,**required_variables):
        
        #len(self.SNR.shape[1])
        if len(self.SNR)>0:
            self._prev_val=self.SNR[-1]
            self._this_order_num = self.SNR.shape[0]+1
        else:
            self.this_order_num = 1
            # to handle the first parameter when _pre_val is the pre set int
            if isinstance(self.n_gauss_pulse, np.ndarray):
                self._prev_val = self.minimum * np.ones(self.n_gauss_pulse.shape)
        try:
            self._tot_order_num=self.n_gauss_pulse.astype(int)
        except:
            self._tot_order_num=int(self.n_gauss_pulse)
        return dict(_prev_val=self._prev_val,_this_order_num=self._this_order_num, _tot_order_num=self._tot_order_num)

In [10]:
priors = bilby.core.prior.dict.ConditionalPriorDict()
priors = tbilby.core.base.create_transdimensional_priors(transdimensional_prior_class=TransdimensionalConditionalDescendingOrderStatPriorSNR,\
                                                          param_name='SNR',\
                                                          nmax= 6,\
                                                          nested_conditional_transdimensional_params=['SNR'],\
                                                          conditional_transdimensional_params=[],\
                                                          conditional_params=['n_gauss_pulse'],\
                                                          prior_dict_to_add=priors,\
                                                          SaveConditionFunctionsToFile=False,\
                                                          minimum= 0,maximum=10,prev_val=10,this_order_num=1)

priors["mu0"] = bilby.core.prior.Uniform(0, 150, "SNR0")
priors["mu1"] = bilby.core.prior.Uniform(0, 150, "SNR1")
priors["mu2"] = bilby.core.prior.Uniform(0, 150, "SNR2")
priors["mu3"] = bilby.core.prior.Uniform(0, 150, "SNR3")
priors["mu4"] = bilby.core.prior.Uniform(0, 150, "SNR4")
priors["mu5"] = bilby.core.prior.Uniform(0, 150, "SNR5")
priors["sigma_g0"] = bilby.core.prior.Uniform(5, 20, "sigma_g0")
priors["sigma_g1"] = bilby.core.prior.Uniform(5, 20, "sigma_g1")
priors["sigma_g2"] = bilby.core.prior.Uniform(5, 20, "sigma_g2")
priors["sigma_g3"] = bilby.core.prior.Uniform(5, 20, "sigma_g3")
priors["sigma_g4"] = bilby.core.prior.Uniform(5, 20, "sigma_g4")
priors["sigma_g5"] = bilby.core.prior.Uniform(5, 20, "sigma_g5")
priors["sigma"] = 0.15

priors['n_gauss_pulse'] = tbilby.core.prior.DiscreteUniform(1,6,'n_dimension')

In [15]:
result = bilby.core.sampler.run_sampler(
    likelihood,
    priors=priors,
    sampler="dynesty",
    outdir='outdir',
    label='dynesty',
    sample='acceptance-walk',
    nlive= 1000,
    naccept= 60,
    resume=True,
    npool=1,
)